In [ ]:
# adding bytetree filters
def add_cme_expiry(temp):
  # cme expiry
  temp['datetime'] = pd.to_datetime(temp.index)
  # this might get some wrong?
  # temp['cme_expiry'] = (temp.datetime.dt.day > temp.datetime.dt.days_in_month-7) & (temp.datetime.dt.weekday == 4)

  # The final settlement value is based on the CME CF Bitcoin Reference Rate (BRR) at 4:00 p.m. London time on the expiration day of the futures contract.
  # 17:00 SAST, 15:00 UTC
  temp['cme_expiry'] = (temp.datetime.dt.day > temp.datetime.dt.days_in_month-7) & (temp.datetime.dt.weekday == 4) & (temp.datetime.dt.hour == 15)
  # temp['cme_period'] = temp['cme_expiry'].cumsum()
  return temp

def time_since_marker(df, column, cap=None):
    steps = 0
    step_list = []
    for isadj in df[column].tolist():
        steps += 1
        if isadj:
            steps = 0
        if cap and steps > cap:
            steps = 0
        step_list.append(steps)
    return pd.Series(step_list).values

def marker_elapsed(series, cap=None):
    steps = 0
    step_list = []
    for marker in series.tolist():
        steps += 1
        if marker:
            steps = 0
        if cap and steps > cap:
            steps = 0
        step_list.append(steps)
    return pd.Series(step_list).values

DIF_ADJ_BLOCKS = 2016
def add_dif_adj_h(df):
    if 'dif' in df.columns:
        df['DiffChanged'] = abs(df['dif'] - df['dif'].shift(1)) > 0.0
        # 144 should be one days worth of blocks
        df['blockadjustment'] = df['hgt'] % DIF_ADJ_BLOCKS == 0
        print (df.loc[df.DiffChanged, ['DiffChanged', 'blockadjustment', 'time']].tail(500))
    else:
        df_dif = pd.read_csv("./data/btc.com_diff.csv") # , parse_dates=["timestamp"]
        # is currently to the minute, CDD hourly is AM/PM format
        # df_dif["adj_date"] = df_dif.adj_date.dt.strftime("%Y-%m-%d %I:00 %s")
        df_dif["adj_date"] = pd.to_datetime(df_dif.timestamp, unit="s")
        df_dif["adj_date"] = pd.to_datetime(df_dif.adj_date.dt.strftime("%Y-%m-%d %H:00:00"))
        df = df.merge(df_dif, how="left", left_on="datestamp", right_on="adj_date")
        df["blockadjustment"] = pd.notnull(df.change)
        # df["adj_period"] = df["blockadjustment"].cumsum()
    return df

# prep daily bytetree
def add_dif_adj(df):
    df['DiffChanged'] = abs(df['dif'] - df['dif'].shift(1)) > 0.0
    # 144 should be one days worth of blocks
    df['blockadjustment'] = df['hgt'] % DIF_ADJ_BLOCKS < 144
    return df


In [ ]:
# trying to consolidate colab FFT_Norm notebook locally
# !pip install scaleogram
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
from preprocess import normalise_data, detrend, preprocess_frame, load_frame
## from preprocess import add_cme_expiry

# df = pd.read_csv("DATASET.CSV")
df = load_frame()
# df = preprocess_frame(df)
print (df.tail(5))
# print (df.columns)

# fig = px.line(df, x=df.index, y="value", log_y=True)
# fig


In [ ]:
import plotly.express as px
from plotly.subplots import make_subplots

def seasonal_violin(df, x_col, y_col):
    # fig = px.violin(df, y=y_col, x=x_col, box=True, points='all')
    fig = px.box(df, y=y_col, x=x_col) # , points='all'
    return fig

def seasonal_index_plot(df, x_col, y_col, use_violin=False):
    import numpy as np
    if use_violin:
        fig = make_subplots(rows=3, cols=1, shared_xaxes=True)
        fig1 = px.violin(df, y=y_col, x=x_col, box=True, points='all')
        fig.add_trace(fig1['data'][0], row=3, col=1)
    else:
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

    # df_pv = pd.pivot_table(df, index=[x_col], values=[y_col], aggfunc=['mean', 'median', 'std'])
    # https://stackoverflow.com/questions/34260003/pandas-pivot-table-percentile
    def my25(g):
        return np.percentile(g, 25)

    def my75(g):
        return np.percentile(g, 75)

    df_pv = pd.pivot_table(df, index=[x_col], values=[y_col], aggfunc=[np.mean, np.median, np.std, my25, my75])

    mean_line = df_pv[[('median', y_col)]].values
    upper_band = df_pv[[('my75', y_col)]].values
    lower_band = df_pv[[('my25', y_col)]].values

    # mean_line = df_pv[[('mean', y_col)]].values
    # upper_band = mean_line + (df_pv[[('std', y_col)]].values / 2)
    # lower_band = mean_line - (df_pv[[('std', y_col)]].values / 2)
    
    fig.add_scatter(x=df_pv.index, y=mean_line[:, 0], name="mean", row=2, col=1)
    fig.add_scatter(x=df_pv.index, y=upper_band[:, 0], name="upper", row=2, col=1)
    fig.add_scatter(x=df_pv.index, y=lower_band[:, 0], name="lower", row=2, col=1)
    
    # can hide legend in BB, but nice
    fig.add_scatter(x=df_pv.index, y=mean_line[:, 0], name="mean", row=1, col=1)
    
    # fig.show()
    return fig


In [ ]:
# https://www.datacamp.com/community/tutorials/pandas-multi-index

def cluster_plot(df, instance_col, series_col, value_col):
    df["truncdate"] = pd.to_datetime(df["datestamp"]).dt.date

    piv = df.set_index([instance_col, series_col])
    # piv[["cme_expiry"]].to_csv("test.csv")
    piv = piv[[value_col]]
    piv = piv.unstack()
    piv.columns = piv.columns.droplevel(0)
    piv = piv.fillna(0)

    total_clusters = 2
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=total_clusters, random_state=42).fit(piv)
    piv["cluster"] = kmeans.labels_

    INDEX_PLOT = False
    if INDEX_PLOT:
        # labels clusters in df
        for instance in piv.index:
            df.loc[df[instance_col]==instance, "cluster"] = piv.loc[instance, "cluster"]

        for cluster in range(total_clusters):
            cl_df = df[df.cluster==cluster]
            seasonal_index_plot(cl_df, series_col, value_col).show()

    else:
        cl = pd.pivot_table(piv, index="cluster")
        cl = cl.stack().reset_index()
        cl = cl.rename(columns={0: "val"})
        for cluster in range(total_clusters):
            cl_df = cl[cl.cluster==cluster]
            px.line(cl_df, x=series_col, y="val").show()

instance_col = "period"
series_col = "bars_elapsed"

instance_col = "truncdate"
series_col = "hour"
# value_col = "value"
value_col = "percentchange"

cluster_plot(df_m, instance_col, series_col, value_col)

In [ ]:
# event based, TODO split into cells
if True:
    # daily block adjustment
    REC_EVENT_COL = "blockadjustment"
    # VALUE_COL = "percentchange"
    VALUE_COL = "close"
    df_m = add_dif_adj(df)
    CAP = 14
elif False:
    # hourly block adjustment
    REC_EVENT_COL = "blockadjustment"
    # VALUE_COL = "percentchange"
    # VALUE_COL = "returns"
    VALUE_COL = "value"
    df_m = add_dif_adj_h(df)
    # Check this could be hourly/date based or bytreee block based
    CAP = DIF_ADJ_BLOCKS
    CAP = 14*24
elif False:
    # daily cme expiry
    REC_EVENT_COL = "cme_expiry"
    VALUE_COL = "close"
    # VALUE_COL = "percentchange"
    # VALUE_COL = "returns"
    df_m = add_cme_expiry(df)
    # CAP = 31
    CAP = 31 * 24

# CAP = None
if CAP:
    # forward version
    df_m["bars_elapsed"] = marker_elapsed(df_m[REC_EVENT_COL], cap=CAP)
    # backward version
    df_m["tminus_bars_elapsed"] = marker_elapsed(df_m[REC_EVENT_COL][::-1], cap=CAP)[::-1]

    # df_m["bars_elapsed"] = time_since_marker(df_m, REC_EVENT_COL, cap=CAP) # , cap=DIF_ADJ_BLOCKS, cap=15, cap=31
    # set the boolean marker where we are capped as well marked
    df_m.loc[df_m.bars_elapsed == 0, REC_EVENT_COL] = True
    df_m["period"] = df_m[REC_EVENT_COL].cumsum()
else:
    # forward version
    df_m["bars_elapsed"] = marker_elapsed(df_m[REC_EVENT_COL])
    # backward version
    df_m["tminus_bars_elapsed"] = marker_elapsed(df_m[REC_EVENT_COL][::-1])[::-1]
    # print (df_m[["bars_elapsed", "tminus_bars_elapsed"]])

# show marker locations
fig = px.line(x=df_m.index, y=df_m.close, log_y=True)
marks = df_m[df_m[REC_EVENT_COL]==True]
fig.add_scatter(x=marks.index, y=marks.close, mode="markers", marker=dict(size=10, symbol=5, color="green"), name="Marker")
fig.add_scatter(x=df_m.index, y=df_m.bars_elapsed, mode="lines+markers", name="Elapsed")
fig.show()

# df = df[df['bars_elapsed'] < CAP]
# df_m = df_m[pd.notnull(df_m['bars_elapsed'])]

seasonal_index_plot(df_m, x_col="bars_elapsed", y_col=VALUE_COL).show()
seasonal_index_plot(df_m, x_col="tminus_bars_elapsed", y_col=VALUE_COL).show()

In [ ]:
# use markers in other notebooks
df_m.to_csv("dataset.csv", index=False)

In [ ]:
# hourly kill zones
# VALUE_COL = "percentchange"
# VALUE_COL = "returns"
VALUE_COL = "close"
seasonal_index_plot(df, x_col="hour", y_col=VALUE_COL).show()
seasonal_violin(df, x_col="hour", y_col=VALUE_COL).show()

# TODO: K-Means groups, df.T inside of the pivot_table?
# cluster_groups(df, x_col="Hour", y_col=VALUE_COL).show()

In [ ]:
# minute resolution kill zones
VALUE_COL = "close"
df["timestamp"] = df["datestamp"].dt.strftime("%H:%M")
seasonal_index_plot(df, x_col="timestamp", y_col=VALUE_COL).show()

In [ ]:
# annual seasonal
seasonal_index_plot(df, x_col="doy", y_col="value").show()

In [ ]:
# seasonal_index_plot(df, x_col="dayname", y_col="close").show()
# Monday=0, Sunday=6
seasonal_index_plot(df, x_col="dow", y_col="close").show()
# seasonal_index_plot(df, x_col="dom", y_col="close").show()


In [ ]:
# Date Formatting or Changing Tick labels
# https://plotly.com/python/tick-formatting/
# tickmode = 'array', but still tricky for yearly
# plotting one sample year with date ticks and averages on shared xaxis?

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [(x+1)*30 for x in range(12)],
        ticktext = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    )
)
fig.show()
for x in zip([(x+1)*28 for x in range(12)], ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']):
    print (x)